In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

from fastai import *
from fastai.vision import *
from fastai.vision.gan import *
from ArNet.generators import *
from ArNet.critics import *
from ArNet.dataset import *
from ArNet.loss import *
from ArNet.save import *
from ArNet.fid_loss import *
from ArNet.ssim import *
from ArNet.metrics import *

import torchvision
import geffnet # efficient/ mobile net
from PIL import Image as PilImage
import cv2

In [ ]:
def toEven(sz):
    tempSz = [sz[0], sz[1]]
    if sz[0]%2 != 0:
        tempSz[0] += 1
    if sz[1]%2 != 0:
        tempSz[1] += 1
    return tempSz

In [ ]:
path = Path('./dataset/')

path_fullRes = path/'DIV2K_train_HR'

path_lowRes_512 = path/'DIV2K_train_LR_512_QF20'
path_lowRes_Full = path/'DIV2K_train_LR_Full_QF20'

In [ ]:
model = geffnet.mobilenetv3_rw
loss_func = lpips_loss()

bs=1

data_gen = get_DIV2k_data_QF(path_lowRes_Full, path_fullRes, bs=bs, sz=1024)

learn_gen = gen_learner_wide(data=data_gen,
                             gen_loss=loss_func,
                             arch = model,
                             nf_factor=2)

In [ ]:
weights = "/data/students_home/fmameli/repos/Artifact_Removal_GAN/models/unet_wideNf2_superRes_mobilenetV3_GAN_best"
learn_gen.load(weights, with_opt=False)

In [ ]:
learn_gen.export("/data/students_home/fmameli/repos/Artifact_Removal_GAN/models/standard.pkl" )

In [ ]:
model = geffnet.mobilenetv3_small_minimal_100
loss_func = lpips_loss()

bs=1

data_gen = get_DIV2k_data_QF(path_lowRes_Full, path_fullRes, bs=bs, sz=512)

learn_gen = gen_learner_wide(data=data_gen,
                             gen_loss=loss_func,
                             arch = model,
                             nf_factor=2)

In [ ]:
weights = "/data/students_home/fmameli/repos/Artifact_Removal_GAN/wandb/run-20200515_110129-unet_wideNf2_superRes_mobilenetMinimal_1k_P64px_VGG_SuperRes_05-15_13_01/bestmodel"
learn_gen.load(weights, with_opt=False)

In [ ]:
learn_gen.export("/data/students_home/fmameli/repos/Artifact_Removal_GAN/models/minimal.pkl" )

# Minimal model

In [ ]:
root_model_path = Path("/data/students_home/fmameli/repos/Artifact_Removal_GAN/models/")
exported_model_minimal  =Path("/data/students_home/fmameli/repos/Artifact_Removal_GAN/models/minimal.pkl")

In [ ]:
testFrameList = ImageImageList.from_folder(path/'DIV2K_train_LR_512_QF20', presort=True)

In [ ]:
learn_min = load_learner(path=root_model_path, file=exported_model_minimal, test=testFrameList[:24], tfm_y=False)

In [ ]:
learn_min.data.batch_size = 2

In [ ]:
i = 803
id_img = str(i).zfill(4)

img_low = open_image("dataset/DIV2K_train_LR_512_QF20/" + id_img + ".jpg")

In [ ]:
%%timeit -n 1 -r 100
p,img_hr,b = learn_min.predict(img_low)

In [ ]:
%%timeit -n 1 -r 4
p = learn_min.get_preds(ds_type=DatasetType.Test)

# Standard GAN model

In [ ]:
root_model_path = Path("/data/students_home/fmameli/repos/Artifact_Removal_GAN/models/")
exported_model_standard =Path("/data/students_home/fmameli/repos/Artifact_Removal_GAN/models/standard.pkl")

In [ ]:
testFrameList = ImageImageList.from_folder(path/'DIV2K_train_LR_512_QF20', presort=True)

In [ ]:
learn_std = load_learner(path=root_model_path, file=exported_model_standard, test=testFrameList[:24], tfm_y=False)

In [ ]:
learn_std.data.batch_size = 2

In [ ]:
%%timeit -n 1 -r 100
p,img_hr,b = learn_std.predict(img_low)

In [ ]:
%%timeit -n 1 -r 4
p = learn_std.get_preds(ds_type=DatasetType.Test)

### Image processing

In [ ]:
def toEven(sz):
    tempSz = [sz[0], sz[1]]
    if sz[0]%2 != 0:
        tempSz[0] += 1
    if sz[1]%2 != 0:
        tempSz[1] += 1
    return tempSz

In [ ]:
weights = "/data/students_home/fmameli/repos/SuperRes/models/unet_wideNf2_superRes_mobilenetV3_GAN_best"
learn_gen.load(weights, with_opt=False)

for i in range(1, 901):
    id_img = str(i).zfill(4)

    img_low = open_image("dataset/DIV2K_train_LR_1024_QF20/" + id_img + ".jpg")
    size=toEven(img_low.size)
    data_gen = get_DIV2k_data_QF(path_lowRes_Full, path_fullRes, bs=bs, sz=size)

    learn_gen.data = data_gen
    
    p,img_hr,b = learn_gen.predict(img_low)
    p.save("output_imgs/" + id_img + "_GAN.png")
    print("dataset/DIV2K_train_LR_1024_QF20/" + id_img + ".jpg")

In [ ]:
weights = "/data/students_home/fmameli/repos/SuperRes/models/unet_superRes_mobilenetV3_MSE_gen_512px_2"
learn_gen.load(weights, with_opt=False)

for i in range(1, 901):
    id_img = str(i).zfill(4)

    img_low = open_image("dataset/DIV2K_train_LR_1024_QF20/" + id_img + ".jpg")
    size=toEven(img_low.size)
    data_gen = get_DIV2k_data_QF(path_lowRes_Full, path_fullRes, bs=bs, sz=size)

    learn_gen.data = data_gen
    
    p,img_hr,b = learn_gen.predict(img_low)
    p.save("output_imgs/" + id_img + "_MSE.png")
    print("dataset/DIV2K_train_LR_1024_QF20/" + id_img + ".jpg")

In [ ]:
weights = "/data/students_home/fmameli/repos/SuperRes/models/unet_superRes_mobilenetV3_SSIM_gen_512px_0"
learn_gen.load(weights, with_opt=False)

for i in range(1, 901):
    id_img = str(i).zfill(4)

    img_low = open_image("dataset/DIV2K_train_LR_1024_QF20/" + id_img + ".jpg")
    size=toEven(img_low.size)
    data_gen = get_DIV2k_data_QF(path_lowRes_Full, path_fullRes, bs=bs, sz=size)

    learn_gen.data = data_gen
    
    p,img_hr,b = learn_gen.predict(img_low)
    p.save("output_imgs/" + id_img + "_SSIM.png")
    print("dataset/DIV2K_train_LR_1024_QF20/" + id_img + ".jpg")

In [ ]:
weights = "/data/students_home/fmameli/repos/SuperRes/models/unet_wideNf2_superRes_mobilenetV3_GAN_best"
learn_gen.load(weights, with_opt=False)

p_gan,img_hr,b = learn_gen.predict(img_low)
p_gan

In [ ]:
weights = "/data/students_home/fmameli/repos/SuperRes/models/unet_superRes_mobilenetV3_MSE_gen_512px_2"
learn_gen.load(weights, with_opt=False)

p_mse,img_hr,b = learn_gen.predict(img_low)
p_mse

In [ ]:
weights = "/data/students_home/fmameli/repos/SuperRes/models/unet_superRes_mobilenetV3_SSIM_gen_512px_0"
learn_gen.load(weights, with_opt=False)

p_ssim,img_hr,b = learn_gen.predict(img_low)
p_ssim